In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader
import faiss
import numpy as np
import re
from IPython.display import Markdown

In [12]:
# PDFを読み込む関数
def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# テキストをチャンクに分割する関数
def split_text(text, chunk_size=300):
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

# PDFから抽出したテキストから改行コードを削除する関数
def clean_text(text):
    return text.replace("\n", "").strip()

In [13]:
# PDFファイルをロードしてチャンクに分割
file_path = "../input/MIハンドブック_はじめに.pdf"
text = load_pdf(file_path)
cleaned_text = clean_text(text)
documents = split_text(cleaned_text)

In [14]:
documents

['はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを',
 '活用して 材料開発を加速したい。 」という気持ち だと考えています 。   著者の高原と福岡は 多くの民間企業などに対して MI活用の支援をさせて頂いています。 学生時代に は二人とも 材料工学を専攻 し、それぞれ別 メーカーでMIを活用した材料開発業務を経て、 現在はコンサルティング ・データ分析・講演・教育・データ活用システムの提供 といった 様々な立場 ・内容で活動を行って います。  MIの普及に伴い、様々な書籍やセミナーが 発刊・実施されています 。しかしそれは、MIとは？という概論か、 整形されたデータに対して分析アルゴリズムを適用する ものが主と感じています。  材料開発の実務の',
 '中で MIを活用するには、さらに踏み込んだ情報が必要です。 例えば、データの整形方法。 MIに限らず機械学習 ・データ分析の入門書では分析可能 な形に整形されたデータから分析がスタートします。一方で、 MIが普及していない 材料開発の現場では、 データ分析フレンドリーな 形でデータが蓄積されていません。そのため、そ のデータを分析可能にする手順が分からずMIの活用を断念してしまうことがあります 。 次にデータや分析結果の 見方という観点。 本書執筆の 時点では 、AIにデータを丸ごと投げ込み、出てきた結果をそのまま実験すればより良い材料が出来るということは 、ほとんどの場合 ありません 。手持',
 'ちのデータのどのような点に着目し、 データ分析を適用するか。出てきた分析結果を どのように 材料開発の加速に役立て ていくか。こういった ノウハウが必要に なります。

In [15]:
# SentenceTransformerを使用して埋め込みを生成
embedding_model = SentenceTransformer("tohoku-nlp/bert-base-japanese-char")
embeddings = embedding_model.encode(documents, convert_to_tensor=False)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
embeddings

array([[-0.00860138, -0.01544509,  0.05732762, ...,  0.10766538,
         0.02744278,  0.07329488],
       [-0.02874077, -0.03830459,  0.0481893 , ...,  0.10121736,
         0.02751632,  0.04565822],
       [-0.00059892, -0.09614192,  0.06609966, ...,  0.07873933,
        -0.14247668,  0.16064684],
       ...,
       [ 0.05802056, -0.14048971,  0.08772241, ...,  0.11404857,
        -0.02176722,  0.12419258],
       [ 0.050071  , -0.04091918, -0.02482079, ...,  0.0119293 ,
         0.0106582 ,  0.05920728],
       [ 0.02757211,  0.03278049,  0.03972042, ...,  0.04832372,
         0.0965587 , -0.30393988]], dtype=float32)

In [17]:
# FAISSインデックスを作成
d = embeddings[0].shape[0]  # 埋め込みの次元
index = faiss.IndexFlatL2(d)
index.add(np.array(embeddings))

In [23]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-jpn-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-jpn-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [57]:
# テキスト生成パイプラインを設定
text_gen_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, temperature=0.7)

The model 'Gemma2ForCausalLM' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'Qwen2AudioForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


In [63]:
def ask_question(question):
    # 質問を埋め込みに変換
    question_embedding = embedding_model.encode([question], convert_to_tensor=False)
    
    # FAISSで最も近いチャンクを検索
    _, indices = index.search(np.array(question_embedding), k=1)
    retrieved_text = documents[indices[0][0]]
    
    # 質問と関連情報、回答の間に改行を追加
    input_text = f"質問: {question}\n\n関連情報: {retrieved_text}\n\n回答: "
    
    # パイプラインを使用して回答を生成
    outputs = text_gen_pipeline(input_text, max_new_tokens=256)
    
    # リストから文字列を取得
    assistant_response = outputs[0]['generated_text'].strip()
    
    return assistant_response

In [59]:
question = 'MIとは何ですか？'

outputs = text_gen_pipeline(question, max_new_tokens=256)
response = outputs[0]['generated_text'].strip()
print(response)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**


In [60]:
display(Markdown(response))

MIとは何ですか？

「**AI**」の略称で、人工知能の技術です。

**AIとは？**

* **人間のような思考・判断能力を持つコンピュータ**
* **学習・分析・予測・行動**を自動化できる能力を持つ

**AIの活用例**

* **自動運転車**
* **医療診断**
* **顧客対応**
* **商品推薦**
* **翻訳**
* **画像認識**

In [64]:
print(ask_question(question))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

質問: MIとは何ですか？

関連情報: はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを

回答: 

MI（Materials Informatics）は、**材料の特性や性能を理解し、設計・開発を効率化するための、材料科学とデータ分析の融合**です。

**具体的には、以下の要素が含まれます:**

* **材料のデータベース:**  膨大な材料データの蓄積と管理
* **データ分析:**  材料データから、材料の特性や性能を分析
* **AI・機械学習:**  材料の設計・開発を支援するAIや機械学習モデルの開発
* **シミュレーション:**  材料の性能を予測するためのシミュレーション

**MIのメリット:**

* **材料開発の効率化:**  従来の材料開発プロセスを大幅に短縮
* **新しい材料の発見:**  膨大なデータから、新しい材料の発見が可能
* **材料性能の向上:**  材料の性能をより正確に予測・設計
* **コスト削減:**  材料開発コストを削減


In [65]:
display(Markdown(ask_question(question)))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

質問: MIとは何ですか？

関連情報: はじめに   良い材料を 早く創りたいので 、マテリアルズ・インフォマティクス（ MI）を活用して”いる”。このような声 はもう珍しくありません 。 MIブームに端を発して 、大学・企業 ともにMIの活用検討が 始められて 既にある程度の期間が 経過しました 。すでにMIに取り組んでいる 大学・企業 からはMI活用の成果 が多く報告されており 、MIは期待に応えられるということが共通理解になりつつあります。 MIとは？有効なものなのか？が大勢を占めていた数年前とは隔世の感があります。  本書を手に取っていただいた方は、 材料開発者として、 あるいは材料開発を支援する データ分析者として「MIを

回答: 

MI（Materials Informatics）は、**材料の特性や性能を理解し、設計・開発を効率化するための、材料科学とデータ分析の融合**です。

**具体的には、以下の要素が含まれます:**

* **材料のデータベース:**  膨大な材料データの蓄積と管理
* **データ分析:**  材料データから、材料の特性や性能を分析
* **AI・機械学習:**  材料の設計・開発を支援するAIや機械学習モデルの開発
* **シミュレーション:**  材料の性能を予測するためのシミュレーション

**MIのメリット:**

* **材料開発の効率化:**  従来の材料開発プロセスを大幅に短縮
* **新しい材料の発見:**  膨大なデータから、新しい材料の発見が可能
* **材料性能の向上:**  材料の性能をより正確に予測・設計
* **コスト削減:**  材料開発コストを削減